In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
281,"It's too bad these guys, the so-called judges,...",negative
115,A comparison between this movie and 'The Last ...,negative
180,I saw this Australian film about 10 years ago ...,positive
985,Thanks to the BBC for this show. I used to suf...,negative
197,I was not very excited to see this movie in th...,negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
df = normalize_text(df)
df.head()

,review,sentiment
281,bad guy so called judge jerk even nominally se...,negative
115,comparison movie the last detail made some cha...,negative
180,saw australian film year ago never forgotten i...,positive
985,thanks bbc show used suffer inferiority comple...,negative
197,excited see movie first place group u decided ...,negative


In [6]:
df['sentiment'].value_counts()

sentiment
negative    256
positive    244
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
281,bad guy so called judge jerk even nominally se...,0
115,comparison movie the last detail made some cha...,0
180,saw australian film year ago never forgotten i...,1
985,thanks bbc show used suffer inferiority comple...,0
197,excited see movie first place group u decided ...,0


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/BharathkumarK19/Capstone-Project.mlflow')
dagshub.init(repo_owner='BharathkumarK19', repo_name='Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "BharathkumarK19/Capstone-Project"

Repository BharathkumarK19/Capstone-Project initialized!

2025/08/04 18:04:54 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/9e71dcd262e148a589ee72c0a3ec03ec', creation_time=1754310895184, experiment_id='0', last_update_time=1754310895184, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [14]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-08-04 18:05:04,204 - INFO - Starting MLflow run...
2025-08-04 18:05:04,970 - INFO - Logging preprocessing parameters...
2025-08-04 18:05:06,080 - INFO - Initializing Logistic Regression model...
2025-08-04 18:05:06,081 - INFO - Fitting the model...
2025-08-04 18:05:06,112 - INFO - Model training complete.
2025-08-04 18:05:06,114 - INFO - Logging model parameters...
2025-08-04 18:05:06,514 - INFO - Making predictions...
2025-08-04 18:05:06,517 - INFO - Calculating evaluation metrics...
2025-08-04 18:05:06,527 - INFO - Logging evaluation metrics...
2025-08-04 18:05:07,977 - INFO - Saving and logging the model...
2025/08/04 18:05:20 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025-08-04 18:05:20,706 - INFO - Model training and logging completed in 15.74 seconds.
2025-08-04 18:05:20,708 - INFO - Accuracy: 0.6
2025-08-04 18:05:20,709 - INFO - Precision: 0.5625
2025-08-04 18:05:20,7